# Montée en ordre et conditionnement

In [ ]:
import numpy as np
import quadpy
import plotly.graph_objs as go
from scipy.integrate import newton_cotes
from scipy.special import roots_legendre

On considère l'intégration de la fonction $f(x) = \cos(2x)$ sur l'intervalle $[0,1]$ dont le résultat exact est $0.5 \sin(2)$.

In [ ]:
def f(x):
    return np.cos(2*x)

res_exa = 0.5*np.sin(2)

## Formules de Newton-Cotes  

### Poids calculés avec scipy 

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    b, _ = newton_cotes(si-1, equal=1)
    b = b/(si-1)
    c = np.linspace(0, 1, si)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

### Poids calculés avec quadpy

In [ ]:
s = np.array([2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 16, 18, 20, 24, 28, 32, 36, 40])

err = np.zeros(s.size)
res_nc_quadpy = np.zeros(s.size)
b_nc_quadpy = []

for i, si in enumerate(s):
    #print(si, " ", end="")
    quad = quadpy.c1.newton_cotes_closed(si-1)
    b_nc_quadpy.append(0.5*quad.weights)
    c = np.linspace(0, 1, si)
    res_nc_quadpy[i] = np.sum(b_nc_quadpy[i] * f(c))
    err[i] = np.abs(res_nc_quadpy[i] - res_exa)

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

### Perturbation des $f(c_i)$

In [ ]:
eps = 0.01

err = np.zeros(s.size)
err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c = np.linspace(0, 1, si)
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b_nc_quadpy[i] * f_pert)
    err_pert[i] = np.abs(res_nc_quadpy[i] - res_pert)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()

## Formule de Clenshaw-Curtis

In [ ]:
def xcheb(n):
    if n == 0:
        return np.array([])
    else:
        x = 0.5*np.cos( (2*np.arange(0,n)+1)*np.pi / (2*n) ) + 0.5
    return x

def coeffs_clenshawcurtis(n):
    x = xcheb(n+1) # n+1 Chebyshev nodes
    tab_k = np.arange(0, n+1, dtype='float')
    b = 1 / (tab_k + 1)
    M = np.vander(x, increasing=True)
    w = np.linalg.solve(np.transpose(M), b)
    return x, w    

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = coeffs_clenshawcurtis(si-1)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    if (err[i]==0): err[i]=1e-16    

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

### Perturbation des $f(c_i)$

In [ ]:
eps = 0.01

err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = coeffs_clenshawcurtis(si-1)
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b * f_pert)
    err_pert[i] = np.abs(res - res_pert)

fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()

## Formule de Gauss

In [ ]:
s = np.arange(2, 41, 1)

err = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = roots_legendre(si)
    c = 0.5*(c+1)
    b = 0.5*b
    res = np.sum(b * f(c))
    err[i] = np.abs(res - res_exa)
    if (err[i] == 0): err[i]=1e-16
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="Erreur")
fig.show()

### Formule perturbée 

In [ ]:
eps = 0.01

s = np.arange(2, 41, 1)

err = np.zeros(s.size)
err_pert = np.zeros(s.size)

for i, si in enumerate(s):
    c, b = roots_legendre(si)
    c = 0.5*(c+1)
    b = 0.5*b
    res = np.sum(b * f(c))
    f_pert = f(c) + eps*(2*np.random.rand(c.size)-1)*f(c)
    res_pert = np.sum(b * f_pert)
    err_pert[i] = np.abs(res - res_pert)
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=s, y=err_pert, mode='lines+markers'))
fig.update_xaxes(title="Nombre d'étages")
fig.update_yaxes(type="log", exponentformat = 'e', title="|Res. - Res. pert|")
fig.show()